# $H_2$ on backend

In [1]:
# Running this as background process because of long running time

from lib import h2_hamiltonian_parity, make_hydrogen_hea
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit import transpile
import time
from datetime import datetime
from qiskit_algorithms.optimizers import L_BFGS_B
import numpy as np

print()
print("=========================================")
print("H_2 case of 0.735 Angstrom distance with HEA ansatz from literature, Bravyi-Kitaev mapper, unlimiter iterations")
print("=========================================")
print()



H_2 case of 0.735 Angstrom distance with HEA ansatz from literature, Bravyi-Kitaev mapper, unlimiter iterations



In [2]:

backend_name = "ibm_brisbane"
service = QiskitRuntimeService()
backend = service.backend(backend_name)
print("Done getting backend")
print()
print("==============ANSATZ==================")
print()

ansatz = make_hydrogen_hea()
print(ansatz)

print()
print("============HAMILTONIAN==============")
print()

start = time.time()
hamiltonian = h2_hamiltonian_parity(distance=0.735)
end = time.time()
print(f"{0.735} case took time to make:", end - start)
print(hamiltonian)

print()
print("=============TRANSPILE====================")
print()

ansatz = transpile(
    [ansatz],
    optimization_level=1,
    backend=backend,
)[0]

hamiltonian = hamiltonian.apply_layout(ansatz.layout)

estimator = Estimator(mode=backend)
estimator.options.resilience_level = 1
estimator.options.dynamical_decoupling.enable = True
estimator.options.dynamical_decoupling.sequence_type = "XY4"

print("Transpiled ansatz:")
print(ansatz)
print("Transpiled hamiltonian:")
print(hamiltonian)


/tmp/ipykernel_1728714/391600473.py:2: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud" and "local" will be the only valid channels. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService()


Done getting backend

==============ANSATZ==================

     ┌────────┐     
q_0: ┤ Ry(θ1) ├──■──
     ├────────┤┌─┴─┐
q_1: ┤ Ry(θ2) ├┤ X ├
     └────────┘└───┘

============HAMILTONIAN==============

0.735 case took time to make: 0.2584688663482666
SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ', 'XX'],
              coeffs=[-1.05237325+0.j,  0.39793742+0.j, -0.39793742+0.j, -0.0112801 +0.j,
  0.1809312 +0.j])

=============TRANSPILE====================

Transpiled ansatz:
global phase: 0
                   ┌────┐┌────────────┐┌─────────┐                  ┌──────┐»
          q_0 -> 0 ┤ √X ├┤ Rz(θ1 + π) ├┤ Rz(π/2) ├──────────────────┤1     ├»
                   ├────┤├────────────┤├─────────┤┌────┐┌──────────┐│  Ecr │»
          q_1 -> 1 ┤ √X ├┤ Rz(θ2 + π) ├┤ Rz(π/2) ├┤ √X ├┤ Rz(-π/2) ├┤0     ├»
                   └────┘└────────────┘└─────────┘└────┘└──────────┘└──────┘»
    ancilla_0 -> 2 ─────────────────────────────────────────────────────────»
                                         

In [3]:

print()
print("==============RUN=====================")
print()

optimizer = L_BFGS_B()
initial_point = np.zeros(ansatz.num_parameters)
iter_energies=[]
q_runtimes=[]

def estimate_energy(parameters):
    start = time.time()
    job = estimator.run(
        [(ansatz, hamiltonian, parameters)]
    )
    print(f"{datetime.now()} Job submitted: {job.job_id()}")
    result = job.result()[0].data.evs
    end = time.time()
    print(f"Job time: {end - start}")
    q_runtimes.append(end - start)
    print(job.metrics())
    print(result)
    print(parameters)
    iter_energies.append(result)
    return result

print(datetime.now())
start = time.time()
result = optimizer.minimize(fun=estimate_energy, x0=initial_point)
end = time.time()
print(f"Total time: {end - start}")
print(result)
print("Final energy:", result.fun)
print(iter_energies)
print(q_runtimes)


==============RUN=====================

2025-05-22 14:52:43.133610
2025-05-22 14:52:45.631157 Job submitted: d0qhw704anx0008m0tq0
Job time: 26.33948588371277
{'bss': {'seconds': 13}, 'usage': {'quantum_seconds': 13, 'seconds': 13}, 'timestamps': {'created': '2025-05-22T12:52:44.992Z', 'finished': '2025-05-22T12:53:07.994Z', 'running': '2025-05-22T12:52:47.715Z'}, 'executions': 0, 'num_circuits': 0, 'num_qubits': None, 'circuit_depths': None, 'qiskit_version': 'qiskit_ibm_runtime-0.39.0,qiskit-1.4.3*,qiskit_nature-0.7.2*', 'estimated_start_time': '2025-05-22T12:52:46.238Z', 'estimated_completion_time': '2025-05-22T12:52:59.238Z', 'position_in_queue': None, 'position_in_provider': None}
-1.0412421678047858
[0. 0.]
2025-05-22 14:53:12.040801 Job submitted: d0qhwdh4anx0008m0tqg
Job time: 25.00780177116394
{'bss': {'seconds': 13}, 'usage': {'quantum_seconds': 13, 'seconds': 13}, 'timestamps': {'created': '2025-05-22T12:53:11.192Z', 'finished': '2025-05-22T12:53:33.376Z', 'running': '2025-0

/home/home/Desktop/PLAYGROUND/vqe-analysis-paper/qiskit-env/lib/python3.13/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:876: UserWarning: Your current pending jobs are estimated to consume 13.33325256552087 quantum seconds, but you only have 13 quantum seconds left in your monthly quota; therefore, it is likely this job will be canceled
  warnings.warn(warning_message)


2025-05-22 15:07:09.437959 Job submitted: d0qj2z3qtdr0008gcbg0
Job time: 26.7071590423584
{'bss': {'seconds': 13}, 'usage': {'quantum_seconds': 13, 'seconds': 13}, 'timestamps': {'created': '2025-05-22T13:07:08.669Z', 'finished': '2025-05-22T13:07:32.070Z', 'running': '2025-05-22T13:07:11.716Z'}, 'executions': 0, 'num_circuits': 0, 'num_qubits': None, 'circuit_depths': None, 'qiskit_version': 'qiskit_ibm_runtime-0.39.0,qiskit-1.4.3*,qiskit_nature-0.7.2*', 'estimated_start_time': '2025-05-22T13:07:10.012Z', 'estimated_completion_time': '2025-05-22T13:07:23.012Z', 'position_in_queue': None, 'position_in_provider': None}
-1.047973430008839
[ 2.89188942e-05 -8.87913161e-07]


IBMRuntimeError: 'Failed to run program: \'403 Client Error: Forbidden for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"message":"Job create exceeds open plan job usage limits","code":4317,"solution":"Please wait until the beginning of next month to submit more jobs when your quota will reset.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''